In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW66NT-MakeConcordance-Revelation.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

Revelation01 = [
    ['Revelation','Rev','01','001','1',"The Revelation of Jesus Christ, which God gave unto him, to shew unto his servants things which must shortly come to pass; and he sent and signified \emph{it} by his angel unto his servant John:"],
    ['Revelation','Rev','01','002','2',"Who bare record of the word of God, and of the testimony of Jesus Christ, and of all things that he saw."],
    ['Revelation','Rev','01','003','3',"Blessed \emph{is} he that readeth, and they that hear the words of this prophecy, and keep those things which are written therein: for the time \emph{is} at hand."],
    ['Revelation','Rev','01','004','4',"John to the seven churches which are in Asia: Grace \emph{be} unto you, and peace, from him which is, and which was, and which is to come; and from the seven Spirits which are before his throne;"],
    ['Revelation','Rev','01','005','5',"And from Jesus Christ, \emph{who} \emph{is} the faithful witness, \emph{and} the first begotten of the dead, and the prince of the kings of the earth. Unto him that loved us, and washed us from our sins in his own blood,"],
    ['Revelation','Rev','01','006','6',"And hath made us kings and priests unto God and his Father; to him \emph{be} glory and dominion for ever and ever. Amen."],
    ['Revelation','Rev','01','007','7',"Behold, he cometh with clouds; and every eye shall see him, and they \emph{also} which pierced him: and all kindreds of the earth shall wail because of him. Even so, Amen."],
    ['Revelation','Rev','01','008','8',"I am Alpha and Omega, the beginning and the ending, saith the Lord, which is, and which was, and which is to come, the Almighty."],
    ['Revelation','Rev','01','009','9',"I John, who also am your brother, and companion in tribulation, and in the kingdom and patience of Jesus Christ, was in the isle that is called Patmos, for the word of God, and for the testimony of Jesus Christ."],
    ['Revelation','Rev','01','010','10',"I was in the Spirit on the Lord's day, and heard behind me a great voice, as of a trumpet,"],
    ['Revelation','Rev','01','011','11',"Saying, I am Alpha and Omega, the first and the last: and, What thou seest, write in a book, and send \emph{it} unto the seven churches which are in Asia; unto Ephesus, and unto Smyrna, and unto Pergamos, and unto Thyatira, and unto Sardis, and unto Philadelphia, and unto Laodicea."],
    ['Revelation','Rev','01','012','12',"And I turned to see the voice that spake with me. And being turned, I saw seven golden candlesticks;"],
    ['Revelation','Rev','01','013','13',"And in the midst of the seven candlesticks \emph{one} like unto the Son of man, clothed with a garment down to the foot, and girt about the paps with a golden girdle."],
    ['Revelation','Rev','01','014','14',"His head and \emph{his} hairs \emph{were} white like wool, as white as snow; and his eyes \emph{were} as a flame of fire;"],
    ['Revelation','Rev','01','015','15',"And his feet like unto fine brass, as if they burned in a furnace; and his voice as the sound of many waters."],
    ['Revelation','Rev','01','016','16',"And he had in his right hand seven stars: and out of his mouth went a sharp twoedged sword: and his countenance \emph{was} as the sun shineth in his strength."],
    ['Revelation','Rev','01','017','17',"And when I saw him, I fell at his feet as dead. And he laid his right hand upon me, saying unto me, Fear not; I am the first and the last:"],
    ['Revelation','Rev','01','018','18',"I \emph{am} he that liveth, and was dead; and, behold, I am alive for evermore, Amen; and have the keys of hell and of death."],
    ['Revelation','Rev','01','019','19',"Write the things which thou hast seen, and the things which are, and the things which shall be hereafter;"],
    ['Revelation','Rev','01','020','20',"The mystery of the seven stars which thou sawest in my right hand, and the seven golden candlesticks. The seven stars are the angels of the seven churches: and the seven candlesticks which thou sawest are the seven churches."]
]

Revelation02 = [
    ['Revelation','Rev','02','001','1',"Unto the angel of the church of Ephesus write; These things saith he that holdeth the seven stars in his right hand, who walketh in the midst of the seven golden candlesticks;"],
    ['Revelation','Rev','02','002','2',"I know thy works, and thy labour, and thy patience, and how thou canst not bear them which are evil: and thou hast tried them which say they are apostles, and are not, and hast found them liars:"],
    ['Revelation','Rev','02','003','3',"And hast borne, and hast patience, and for my name's sake hast laboured, and hast not fainted."],
    ['Revelation','Rev','02','004','4',"Nevertheless I have \emph{somewhat} against thee, because thou hast left thy first love."],
    ['Revelation','Rev','02','005','5',"Remember therefore from whence thou art fallen, and repent, and do the first works; or else I will come unto thee quickly, and will remove thy candlestick out of his place, except thou repent."],
    ['Revelation','Rev','02','006','6',"But this thou hast, that thou hatest the deeds of the Nicolaitans, which I also hate."],
    ['Revelation','Rev','02','007','7',"He that hath an ear, let him hear what the Spirit saith unto the churches; To him that overcometh will I give to eat of the tree of life, which is in the midst of the paradise of God."],
    ['Revelation','Rev','02','008','8',"And unto the angel of the church in Smyrna write; These things saith the first and the last, which was dead, and is alive;"],
    ['Revelation','Rev','02','009','9',"I know thy works, and tribulation, and poverty, (but thou art rich) and \emph{I} \emph{know} the blasphemy of them which say they are Jews, and are not, but \emph{are} the synagogue of Satan."],
    ['Revelation','Rev','02','010','10',"Fear none of those things which thou shalt suffer: behold, the devil shall cast \emph{some} of you into prison, that ye may be tried; and ye shall have tribulation ten days: be thou faithful unto death, and I will give thee a crown of life."],
    ['Revelation','Rev','02','011','11',"He that hath an ear, let him hear what the Spirit saith unto the churches; He that overcometh shall not be hurt of the second death."],
    ['Revelation','Rev','02','012','12',"And to the angel of the church in Pergamos write; These things saith he which hath the sharp sword with two edges;"],
    ['Revelation','Rev','02','013','13',"I know thy works, and where thou dwellest, \emph{even} where Satan's seat \emph{is}: and thou holdest fast my name, and hast not denied my faith, even in those days wherein Antipas \emph{was} my faithful martyr, who was slain among you, where Satan dwelleth."],
    ['Revelation','Rev','02','014','14',"But I have a few things against thee, because thou hast there them that hold the doctrine of Balaam, who taught Balac to cast a stumblingblock before the children of Israel, to eat things sacrificed unto idols, and to commit fornication."],
    ['Revelation','Rev','02','015','15',"So hast thou also them that hold the doctrine of the Nicolaitans, which thing I hate."],
    ['Revelation','Rev','02','016','16',"Repent; or else I will come unto thee quickly, and will fight against them with the sword of my mouth."],
    ['Revelation','Rev','02','017','17',"He that hath an ear, let him hear what the Spirit saith unto the churches; To him that overcometh will I give to eat of the hidden manna, and will give him a white stone, and in the stone a new name written, which no man knoweth saving he that receiveth \emph{it}."],
    ['Revelation','Rev','02','018','18',"And unto the angel of the church in Thyatira write; These things saith the Son of God, who hath his eyes like unto a flame of fire, and his feet \emph{are} like fine brass;"],
    ['Revelation','Rev','02','019','19',"I know thy works, and charity, and service, and faith, and thy patience, and thy works; and the last \emph{to} \emph{be} more than the first."],
    ['Revelation','Rev','02','020','20',"Notwithstanding I have a few things against thee, because thou sufferest that woman Jezebel, which calleth herself a prophetess, to teach and to seduce my servants to commit fornication, and to eat things sacrificed unto idols."],
    ['Revelation','Rev','02','021','21',"And I gave her space to repent of her fornication; and she repented not."],
    ['Revelation','Rev','02','022','22',"Behold, I will cast her into a bed, and them that commit adultery with her into great tribulation, except they repent of their deeds."],
    ['Revelation','Rev','02','023','23',"And I will kill her children with death; and all the churches shall know that I am he which searcheth the reins and hearts: and I will give unto every one of you according to your works."],
    ['Revelation','Rev','02','024','24',"But unto you I say, and unto the rest in Thyatira, as many as have not this doctrine, and which have not known the depths of Satan, as they speak; I will put upon you none other burden."],
    ['Revelation','Rev','02','025','25',"But that which ye have \emph{already} hold fast till I come."],
    ['Revelation','Rev','02','026','26',"And he that overcometh, and keepeth my works unto the end, to him will I give power over the nations:"],
    ['Revelation','Rev','02','027','27',"And he shall rule them with a rod of iron; as the vessels of a potter shall they be broken to shivers: even as I received of my Father."],
    ['Revelation','Rev','02','028','28',"And I will give him the morning star."],
    ['Revelation','Rev','02','029','29',"He that hath an ear, let him hear what the Spirit saith unto the churches."]
]

Revelation03 = [
    ['Revelation','Rev','03','001','1',"And unto the angel of the church in Sardis write; These things saith he that hath the seven Spirits of God, and the seven stars; I know thy works, that thou hast a name that thou livest, and art dead."],
['Revelation','Rev','03','002','2',"Be watchful, and strengthen the things which remain, that are ready to die: for I have not found thy works perfect before God."],
['Revelation','Rev','03','003','3',"Remember therefore how thou hast received and heard, and hold fast, and repent. If therefore thou shalt not watch, I will come on thee as a thief, and thou shalt not know what hour I will come upon thee."],
['Revelation','Rev','03','004','4',"Thou hast a few names even in Sardis which have not defiled their garments; and they shall walk with me in white: for they are worthy."],
['Revelation','Rev','03','005','5',"He that overcometh, the same shall be clothed in white raiment; and I will not blot out his name out of the book of life, but I will confess his name before my Father, and before his angels."],
['Revelation','Rev','03','006','6',"He that hath an ear, let him hear what the Spirit saith unto the churches."],
['Revelation','Rev','03','007','7',"And to the angel of the church in Philadelphia write; These things saith he that is holy, he that is true, he that hath the key of David, he that openeth, and no man shutteth; and shutteth, and no man openeth;"],
['Revelation','Rev','03','008','8',"I know thy works: behold, I have set before thee an open door, and no man can shut it: for thou hast a little strength, and hast kept my word, and hast not denied my name."],
['Revelation','Rev','03','009','9',"Behold, I will make them of the synagogue of Satan, which say they are Jews, and are not, but do lie; behold, I will make them to come and worship before thy feet, and to know that I have loved thee."],
['Revelation','Rev','03','010','10',"Because thou hast kept the word of my patience, I also will keep thee from the hour of temptation, which shall come upon all the world, to try them that dwell upon the earth."],
['Revelation','Rev','03','011','11',"Behold, I come quickly: hold that fast which thou hast, that no man take thy crown."],
['Revelation','Rev','03','012','12',"Him that overcometh will I make a pillar in the temple of my God, and he shall go no more out: and I will write upon him the name of my God, and the name of the city of my God, \emph{which} \emph{is} new Jerusalem, which cometh down out of heaven from my God: and \emph{I} \emph{will} \emph{write} \emph{upon} \emph{him} my new name."],
['Revelation','Rev','03','013','13',"He that hath an ear, let him hear what the Spirit saith unto the churches."],
['Revelation','Rev','03','014','14',"And unto the angel of the church of the Laodiceans write; These things saith the Amen, the faithful and true witness, the beginning of the creation of God;"],
['Revelation','Rev','03','015','15',"I know thy works, that thou art neither cold nor hot: I would thou wert cold or hot."],
['Revelation','Rev','03','016','16',"So then because thou art lukewarm, and neither cold nor hot, I will spue thee out of my mouth."],
['Revelation','Rev','03','017','17',"Because thou sayest, I am rich, and increased with goods, and have need of nothing; and knowest not that thou art wretched, and miserable, and poor, and blind, and naked:"],
['Revelation','Rev','03','018','18',"I counsel thee to buy of me gold tried in the fire, that thou mayest be rich; and white raiment, that thou mayest be clothed, and \emph{that} the shame of thy nakedness do not appear; and anoint thine eyes with eyesalve, that thou mayest see."],
['Revelation','Rev','03','019','19',"As many as I love, I rebuke and chasten: be zealous therefore, and repent."],
['Revelation','Rev','03','020','20',"Behold, I stand at the door, and knock: if any man hear my voice, and open the door, I will come in to him, and will sup with him, and he with me."],
['Revelation','Rev','03','021','21',"To him that overcometh will I grant to sit with me in my throne, even as I also overcame, and am set down with my Father in his throne."],
['Revelation','Rev','03','022','22',"He that hath an ear, let him hear what the Spirit saith unto the churches."]
]

Revelation04 = [
    ['Revelation','Rev','04','001','1',"After this I looked, and, behold, a door \emph{was} opened in heaven: and the first voice which I heard \emph{was} as it were of a trumpet talking with me; which said, Come up hither, and I will shew thee things which must be hereafter."],
['Revelation','Rev','04','002','2',"And immediately I was in the spirit: and, behold, a throne was set in heaven, and \emph{one} sat on the throne."],
['Revelation','Rev','04','003','3',"And he that sat was to look upon like a jasper and a sardine stone: and \emph{there} \emph{was} a rainbow round about the throne, in sight like unto an emerald."],
['Revelation','Rev','04','004','4',"And round about the throne \emph{were} four and twenty seats: and upon the seats I saw four and twenty elders sitting, clothed in white raiment; and they had on their heads crowns of gold."],
['Revelation','Rev','04','005','5',"And out of the throne proceeded lightnings and thunderings and voices: and \emph{there} \emph{were} seven lamps of fire burning before the throne, which are the seven Spirits of God."],
['Revelation','Rev','04','006','6',"And before the throne \emph{there} \emph{was} a sea of glass like unto crystal: and in the midst of the throne, and round about the throne, \emph{were} four beasts full of eyes before and behind."],
['Revelation','Rev','04','007','7',"And the first beast \emph{was} like a lion, and the second beast like a calf, and the third beast had a face as a man, and the fourth beast \emph{was} like a flying eagle."],
['Revelation','Rev','04','008','8',"And the four beasts had each of them six wings about \emph{him}; and \emph{they} \emph{were} full of eyes within: and they rest not day and night, saying, Holy, holy, holy, Lord God Almighty, which was, and is, and is to come."],
['Revelation','Rev','04','009','9',"And when those beasts give glory and honour and thanks to him that sat on the throne, who liveth for ever and ever,"],
['Revelation','Rev','04','010','10',"The four and twenty elders fall down before him that sat on the throne, and worship him that liveth for ever and ever, and cast their crowns before the throne, saying,"],
['Revelation','Rev','04','011','11',"Thou art worthy, O Lord, to receive glory and honour and power: for thou hast created all things, and for thy pleasure they are and were created."]
]

Revelation05 = [
    ['Revelation','Rev','05','001','1',"And I saw in the right hand of him that sat on the throne a book written within and on the backside, sealed with seven seals."],
    ['Revelation','Rev','05','002','2',"And I saw a strong angel proclaiming with a loud voice, Who is worthy to open the book, and to loose the seals thereof?"],
    ['Revelation','Rev','05','003','3',"And no man in heaven, nor in earth, neither under the earth, was able to open the book, neither to look thereon."],
    ['Revelation','Rev','05','004','4',"And I wept much, because no man was found worthy to open and to read the book, neither to look thereon."],
    ['Revelation','Rev','05','005','5',"And one of the elders saith unto me, Weep not: behold, the Lion of the tribe of Juda, the Root of David, hath prevailed to open the book, and to loose the seven seals thereof."],
    ['Revelation','Rev','05','006','6',"And I beheld, and, lo, in the midst of the throne and of the four beasts, and in the midst of the elders, stood a Lamb as it had been slain, having seven horns and seven eyes, which are the seven Spirits of God sent forth into all the earth."],
    ['Revelation','Rev','05','007','7',"And he came and took the book out of the right hand of him that sat upon the throne."],
    ['Revelation','Rev','05','008','8',"And when he had taken the book, the four beasts and four \emph{and} twenty elders fell down before the Lamb, having every one of them harps, and golden vials full of odours, which are the prayers of saints."],
    ['Revelation','Rev','05','009','9',"And they sung a new song, saying, Thou art worthy to take the book, and to open the seals thereof: for thou wast slain, and hast redeemed us to God by thy blood out of every kindred, and tongue, and people, and nation;"],
    ['Revelation','Rev','05','010','10',"And hast made us unto our God kings and priests: and we shall reign on the earth."],
    ['Revelation','Rev','05','011','11',"And I beheld, and I heard the voice of many angels round about the throne and the beasts and the elders: and the number of them was ten thousand times ten thousand, and thousands of thousands;"],
    ['Revelation','Rev','05','012','12',"Saying with a loud voice, Worthy is the Lamb that was slain to receive power, and riches, and wisdom, and strength, and honour, and glory, and blessing."],
    ['Revelation','Rev','05','013','13',"And every creature which is in heaven, and on the earth, and under the earth, and such as are in the sea, and all that are in them, heard I saying, Blessing, and honour, and glory, and power, \emph{be} unto him that sitteth upon the throne, and unto the Lamb for ever and ever."],
    ['Revelation','Rev','05','014','14',"And the four beasts said, Amen. And the four \emph{and} twenty elders fell down and worshipped him that liveth for ever and ever."]
]

Revelation06 = [
    ['Revelation','Rev','06','001','1',"And I saw when the Lamb opened one of the seals, and I heard, as it were the noise of thunder, one of the four beasts saying, Come and see."],
    ['Revelation','Rev','06','002','2',"And I saw, and behold a white horse: and he that sat on him had a bow; and a crown was given unto him: and he went forth conquering, and to conquer."],
    ['Revelation','Rev','06','003','3',"And when he had opened the second seal, I heard the second beast say, Come and see."],
    ['Revelation','Rev','06','004','4',"And there went out another horse \emph{that} \emph{was} red: and \emph{power} was given to him that sat thereon to take peace from the earth, and that they should kill one another: and there was given unto him a great sword."],
    ['Revelation','Rev','06','005','5',"And when he had opened the third seal, I heard the third beast say, Come and see. And I beheld, and lo a black horse; and he that sat on him had a pair of balances in his hand."],
    ['Revelation','Rev','06','006','6',"And I heard a voice in the midst of the four beasts say, A measure of wheat for a penny, and three measures of barley for a penny; and \emph{see} thou hurt not the oil and the wine."],
    ['Revelation','Rev','06','007','7',"And when he had opened the fourth seal, I heard the voice of the fourth beast say, Come and see."],
    ['Revelation','Rev','06','008','8',"And I looked, and behold a pale horse: and his name that sat on him was Death, and Hell followed with him. And power was given unto them over the fourth part of the earth, to kill with sword, and with hunger, and with death, and with the beasts of the earth."],
    ['Revelation','Rev','06','009','9',"And when he had opened the fifth seal, I saw under the altar the souls of them that were slain for the word of God, and for the testimony which they held:"],
    ['Revelation','Rev','06','010','10',"And they cried with a loud voice, saying, How long, O Lord, holy and true, dost thou not judge and avenge our blood on them that dwell on the earth?"],
    ['Revelation','Rev','06','011','11',"And white robes were given unto every one of them; and it was said unto them, that they should rest yet for a little season, until their fellowservants also and their brethren, that should be killed as they \emph{were}, should be fulfilled."],
    ['Revelation','Rev','06','012','12',"And I beheld when he had opened the sixth seal, and, lo, there was a great earthquake; and the sun became black as sackcloth of hair, and the moon became as blood;"],
    ['Revelation','Rev','06','013','13',"And the stars of heaven fell unto the earth, even as a fig tree casteth her untimely figs, when she is shaken of a mighty wind."],
    ['Revelation','Rev','06','014','14',"And the heaven departed as a scroll when it is rolled together; and every mountain and island were moved out of their places."],
    ['Revelation','Rev','06','015','15',"And the kings of the earth, and the great men, and the rich men, and the chief captains, and the mighty men, and every bondman, and every free man, hid themselves in the dens and in the rocks of the mountains;"],
    ['Revelation','Rev','06','016','16',"And said to the mountains and rocks, Fall on us, and hide us from the face of him that sitteth on the throne, and from the wrath of the Lamb:"],
    ['Revelation','Rev','06','017','17',"For the great day of his wrath is come; and who shall be able to stand?"]
]

Revelation07 = [
    ['Revelation','Rev','07','001','1',"And after these things I saw four angels standing on the four corners of the earth, holding the four winds of the earth, that the wind should not blow on the earth, nor on the sea, nor on any tree."],
['Revelation','Rev','07','002','2',"And I saw another angel ascending from the east, having the seal of the living God: and he cried with a loud voice to the four angels, to whom it was given to hurt the earth and the sea,"],
['Revelation','Rev','07','003','3',"Saying, Hurt not the earth, neither the sea, nor the trees, till we have sealed the servants of our God in their foreheads."],
['Revelation','Rev','07','004','4',"And I heard the number of them which were sealed: \emph{and} \emph{there} \emph{were} sealed an hundred \emph{and} forty \emph{and} four thousand of all the tribes of the children of Israel."],
['Revelation','Rev','07','005','5',"Of the tribe of Juda \emph{were} sealed twelve thousand. Of the tribe of Reuben \emph{were} sealed twelve thousand. Of the tribe of Gad \emph{were} sealed twelve thousand."],
['Revelation','Rev','07','006','6',"Of the tribe of Aser \emph{were} sealed twelve thousand. Of the tribe of Nepthalim \emph{were} sealed twelve thousand. Of the tribe of Manasses \emph{were} sealed twelve thousand."],
['Revelation','Rev','07','007','7',"Of the tribe of Simeon \emph{were} sealed twelve thousand. Of the tribe of Levi \emph{were} sealed twelve thousand. Of the tribe of Issachar \emph{were} sealed twelve thousand."],
['Revelation','Rev','07','008','8',"Of the tribe of Zabulon \emph{were} sealed twelve thousand. Of the tribe of Joseph \emph{were} sealed twelve thousand. Of the tribe of Benjamin \emph{were} sealed twelve thousand."],
['Revelation','Rev','07','009','9',"After this I beheld, and, lo, a great multitude, which no man could number, of all nations, and kindreds, and people, and tongues, stood before the throne, and before the Lamb, clothed with white robes, and palms in their hands;"],
['Revelation','Rev','07','010','10',"And cried with a loud voice, saying, Salvation to our God which sitteth upon the throne, and unto the Lamb."],
['Revelation','Rev','07','011','11',"And all the angels stood round about the throne, and \emph{about} the elders and the four beasts, and fell before the throne on their faces, and worshipped God,"],
['Revelation','Rev','07','012','12',"Saying, Amen: Blessing, and glory, and wisdom, and thanksgiving, and honour, and power, and might, \emph{be} unto our God for ever and ever. Amen."],
['Revelation','Rev','07','013','13',"And one of the elders answered, saying unto me, What are these which are arrayed in white robes? and whence came they?"],
['Revelation','Rev','07','014','14',"And I said unto him, Sir, thou knowest. And he said to me, These are they which came out of great tribulation, and have washed their robes, and made them white in the blood of the Lamb."],
['Revelation','Rev','07','015','15',"Therefore are they before the throne of God, and serve him day and night in his temple: and he that sitteth on the throne shall dwell among them."],
['Revelation','Rev','07','016','16',"They shall hunger no more, neither thirst any more; neither shall the sun light on them, nor any heat."],
['Revelation','Rev','07','017','17',"For the Lamb which is in the midst of the throne shall feed them, and shall lead them unto living fountains of waters: and God shall wipe away all tears from their eyes."]
]

Revelation08 = [
    ['Revelation','Rev','08','001','1',"And when he had opened the seventh seal, there was silence in heaven about the space of half an hour."],
['Revelation','Rev','08','002','2',"And I saw the seven angels which stood before God; and to them were given seven trumpets."],
['Revelation','Rev','08','003','3',"And another angel came and stood at the altar, having a golden censer; and there was given unto him much incense, that he should offer \emph{it} with the prayers of all saints upon the golden altar which was before the throne."],
['Revelation','Rev','08','004','4',"And the smoke of the incense, \emph{which} \emph{came} with the prayers of the saints, ascended up before God out of the angel's hand."],
['Revelation','Rev','08','005','5',"And the angel took the censer, and filled it with fire of the altar, and cast \emph{it} into the earth: and there were voices, and thunderings, and lightnings, and an earthquake."],
['Revelation','Rev','08','006','6',"And the seven angels which had the seven trumpets prepared themselves to sound."],
['Revelation','Rev','08','007','7',"The first angel sounded, and there followed hail and fire mingled with blood, and they were cast upon the earth: and the third part of trees was burnt up, and all green grass was burnt up."],
['Revelation','Rev','08','008','8',"And the second angel sounded, and as it were a great mountain burning with fire was cast into the sea: and the third part of the sea became blood;"],
['Revelation','Rev','08','009','9',"And the third part of the creatures which were in the sea, and had life, died; and the third part of the ships were destroyed."],
['Revelation','Rev','08','010','10',"And the third angel sounded, and there fell a great star from heaven, burning as it were a lamp, and it fell upon the third part of the rivers, and upon the fountains of waters;"],
['Revelation','Rev','08','011','11',"And the name of the star is called Wormwood: and the third part of the waters became wormwood; and many men died of the waters, because they were made bitter."],
['Revelation','Rev','08','012','12',"And the fourth angel sounded, and the third part of the sun was smitten, and the third part of the moon, and the third part of the stars; so as the third part of them was darkened, and the day shone not for a third part of it, and the night likewise."],
['Revelation','Rev','08','013','13',"And I beheld, and heard an angel flying through the midst of heaven, saying with a loud voice, Woe, woe, woe, to the inhabiters of the earth by reason of the other voices of the trumpet of the three angels, which are yet to sound!"]
]

Revelation09 = [
['Revelation','Rev','09','001','1',"And the fifth angel sounded, and I saw a star fall from heaven unto the earth: and to him was given the key of the bottomless pit."],
['Revelation','Rev','09','002','2',"And he opened the bottomless pit; and there arose a smoke out of the pit, as the smoke of a great furnace; and the sun and the air were darkened by reason of the smoke of the pit."],
['Revelation','Rev','09','003','3',"And there came out of the smoke locusts upon the earth: and unto them was given power, as the scorpions of the earth have power."],
['Revelation','Rev','09','004','4',"And it was commanded them that they should not hurt the grass of the earth, neither any green thing, neither any tree; but only those men which have not the seal of God in their foreheads."],
['Revelation','Rev','09','005','5',"And to them it was given that they should not kill them, but that they should be tormented five months: and their torment \emph{was} as the torment of a scorpion, when he striketh a man."],
['Revelation','Rev','09','006','6',"And in those days shall men seek death, and shall not find it; and shall desire to die, and death shall flee from them."],
['Revelation','Rev','09','007','7',"And the shapes of the locusts \emph{were} like unto horses prepared unto battle; and on their heads \emph{were} as it were crowns like gold, and their faces \emph{were} as the faces of men."],
['Revelation','Rev','09','008','8',"And they had hair as the hair of women, and their teeth were as \emph{the} \emph{teeth} of lions."],
['Revelation','Rev','09','009','9',"And they had breastplates, as it were breastplates of iron; and the sound of their wings \emph{was} as the sound of chariots of many horses running to battle."],
['Revelation','Rev','09','010','10',"And they had tails like unto scorpions, and there were stings in their tails: and their power \emph{was} to hurt men five months."],
['Revelation','Rev','09','011','11',"And they had a king over them, \emph{which} \emph{is} the angel of the bottomless pit, whose name in the Hebrew tongue \emph{is} Abaddon, but in the Greek tongue hath \emph{his} name Apollyon."],
['Revelation','Rev','09','012','12',"One woe is past; \emph{and}, behold, there come two woes more hereafter."],
['Revelation','Rev','09','013','13',"And the sixth angel sounded, and I heard a voice from the four horns of the golden altar which is before God,"],
['Revelation','Rev','09','014','14',"Saying to the sixth angel which had the trumpet, Loose the four angels which are bound in the great river Euphrates."],
['Revelation','Rev','09','015','15',"And the four angels were loosed, which were prepared for an hour, and a day, and a month, and a year, for to slay the third part of men."],
['Revelation','Rev','09','016','16',"And the number of the army of the horsemen \emph{were} two hundred thousand thousand: and I heard the number of them."],
['Revelation','Rev','09','017','17',"And thus I saw the horses in the vision, and them that sat on them, having breastplates of fire, and of jacinth, and brimstone: and the heads of the horses \emph{were} as the heads of lions; and out of their mouths issued fire and smoke and brimstone."],
['Revelation','Rev','09','018','18',"By these three was the third part of men killed, by the fire, and by the smoke, and by the brimstone, which issued out of their mouths."],
['Revelation','Rev','09','019','19',"For their power is in their mouth, and in their tails: for their tails \emph{were} like unto serpents, and had heads, and with them they do hurt."],
['Revelation','Rev','09','020','20',"And the rest of the men which were not killed by these plagues yet repented not of the works of their hands, that they should not worship devils, and idols of gold, and silver, and brass, and stone, and of wood: which neither can see, nor hear, nor walk:"],
['Revelation','Rev','09','021','21',"Neither repented they of their murders, nor of their sorceries, nor of their fornication, nor of their thefts."]
]

Revelation10 = [
    ['Revelation','Rev','10','001','1',"And I saw another mighty angel come down from heaven, clothed with a cloud: and a rainbow \emph{was} upon his head, and his face \emph{was} as it were the sun, and his feet as pillars of fire:"],
['Revelation','Rev','10','002','2',"And he had in his hand a little book open: and he set his right foot upon the sea, and \emph{his} left \emph{foot} on the earth,"],
['Revelation','Rev','10','003','3',"And cried with a loud voice, as \emph{when} a lion roareth: and when he had cried, seven thunders uttered their voices."],
['Revelation','Rev','10','004','4',"And when the seven thunders had uttered their voices, I was about to write: and I heard a voice from heaven saying unto me, Seal up those things which the seven thunders uttered, and write them not."],
['Revelation','Rev','10','005','5',"And the angel which I saw stand upon the sea and upon the earth lifted up his hand to heaven,"],
['Revelation','Rev','10','006','6',"And sware by him that liveth for ever and ever, who created heaven, and the things that therein are, and the earth, and the things that therein are, and the sea, and the things which are therein, that there should be time no longer:"],
['Revelation','Rev','10','007','7',"But in the days of the voice of the seventh angel, when he shall begin to sound, the mystery of God should be finished, as he hath declared to his servants the prophets."],
['Revelation','Rev','10','008','8',"And the voice which I heard from heaven spake unto me again, and said, Go \emph{and} take the little book which is open in the hand of the angel which standeth upon the sea and upon the earth."],
['Revelation','Rev','10','009','9',"And I went unto the angel, and said unto him, Give me the little book. And he said unto me, Take \emph{it}, and eat it up; and it shall make thy belly bitter, but it shall be in thy mouth sweet as honey."],
['Revelation','Rev','10','010','10',"And I took the little book out of the angel's hand, and ate it up; and it was in my mouth sweet as honey: and as soon as I had eaten it, my belly was bitter."],
['Revelation','Rev','10','011','11',"And he said unto me, Thou must prophesy again before many peoples, and nations, and tongues, and kings."]
]

Revelation11 = [
    ['Revelation','Rev','11','001','1',"And there was given me a reed like unto a rod: and the angel stood, saying, Rise, and measure the temple of God, and the altar, and them that worship therein."],
    ['Revelation','Rev','11','002','2',"But the court which is without the temple leave out, and measure it not; for it is given unto the Gentiles: and the holy city shall they tread under foot forty \emph{and} two months."],
    ['Revelation','Rev','11','003','3',"And I will give \emph{power} unto my two witnesses, and they shall prophesy a thousand two hundred \emph{and} threescore days, clothed in sackcloth."],
    ['Revelation','Rev','11','004','4',"These are the two olive trees, and the two candlesticks standing before the God of the earth."],
    ['Revelation','Rev','11','005','5',"And if any man will hurt them, fire proceedeth out of their mouth, and devoureth their enemies: and if any man will hurt them, he must in this manner be killed."],
    ['Revelation','Rev','11','006','6',"These have power to shut heaven, that it rain not in the days of their prophecy: and have power over waters to turn them to blood, and to smite the earth with all plagues, as often as they will."],
    ['Revelation','Rev','11','007','7',"And when they shall have finished their testimony, the beast that ascendeth out of the bottomless pit shall make war against them, and shall overcome them, and kill them."],
    ['Revelation','Rev','11','008','8',"And their dead bodies \emph{shall} \emph{lie} in the street of the great city, which spiritually is called Sodom and Egypt, where also our Lord was crucified."],
    ['Revelation','Rev','11','009','9',"And they of the people and kindreds and tongues and nations shall see their dead bodies three days and an half, and shall not suffer their dead bodies to be put in graves."],
    ['Revelation','Rev','11','010','10',"And they that dwell upon the earth shall rejoice over them, and make merry, and shall send gifts one to another; because these two prophets tormented them that dwelt on the earth."],
    ['Revelation','Rev','11','011','11',"And after three days and an half the Spirit of life from God entered into them, and they stood upon their feet; and great fear fell upon them which saw them."],
    ['Revelation','Rev','11','012','12',"And they heard a great voice from heaven saying unto them, Come up hither. And they ascended up to heaven in a cloud; and their enemies beheld them."],
    ['Revelation','Rev','11','013','13',"And the same hour was there a great earthquake, and the tenth part of the city fell, and in the earthquake were slain of men seven thousand: and the remnant were affrighted, and gave glory to the God of heaven."],
    ['Revelation','Rev','11','014','14',"The second woe is past; \emph{and}, behold, the third woe cometh quickly."],
    ['Revelation','Rev','11','015','15',"And the seventh angel sounded; and there were great voices in heaven, saying, The kingdoms of this world are become \emph{the} \emph{kingdoms} of our Lord, and of his Christ; and he shall reign for ever and ever."],
    ['Revelation','Rev','11','016','16',"And the four and twenty elders, which sat before God on their seats, fell upon their faces, and worshipped God,"],
    ['Revelation','Rev','11','017','17',"Saying, We give thee thanks, O Lord God Almighty, which art, and wast, and art to come; because thou hast taken to thee thy great power, and hast reigned."],
    ['Revelation','Rev','11','018','18',"And the nations were angry, and thy wrath is come, and the time of the dead, that they should be judged, and that thou shouldest give reward unto thy servants the prophets, and to the saints, and them that fear thy name, small and great; and shouldest destroy them which destroy the earth."],
    ['Revelation','Rev','11','019','19',"And the temple of God was opened in heaven, and there was seen in his temple the ark of his testament: and there were lightnings, and voices, and thunderings, and an earthquake, and great hail."]\
]

Revelation12 = [
    ['Revelation','Rev','12','001','1',"And there appeared a great wonder in heaven; a woman clothed with the sun, and the moon under her feet, and upon her head a crown of twelve stars:"],
['Revelation','Rev','12','002','2',"And she being with child cried, travailing in birth, and pained to be delivered."],
['Revelation','Rev','12','003','3',"And there appeared another wonder in heaven; and behold a great red dragon, having seven heads and ten horns, and seven crowns upon his heads."],
['Revelation','Rev','12','004','4',"And his tail drew the third part of the stars of heaven, and did cast them to the earth: and the dragon stood before the woman which was ready to be delivered, for to devour her child as soon as it was born."],
['Revelation','Rev','12','005','5',"And she brought forth a man child, who was to rule all nations with a rod of iron: and her child was caught up unto God, and \emph{to} his throne."],
['Revelation','Rev','12','006','6',"And the woman fled into the wilderness, where she hath a place prepared of God, that they should feed her there a thousand two hundred \emph{and} threescore days."],
['Revelation','Rev','12','007','7',"And there was war in heaven: Michael and his angels fought against the dragon; and the dragon fought and his angels,"],
['Revelation','Rev','12','008','8',"And prevailed not; neither was their place found any more in heaven."],
['Revelation','Rev','12','009','9',"And the great dragon was cast out, that old serpent, called the Devil, and Satan, which deceiveth the whole world: he was cast out into the earth, and his angels were cast out with him."],
['Revelation','Rev','12','010','10',"And I heard a loud voice saying in heaven, Now is come salvation, and strength, and the kingdom of our God, and the power of his Christ: for the accuser of our brethren is cast down, which accused them before our God day and night."],
['Revelation','Rev','12','011','11',"And they overcame him by the blood of the Lamb, and by the word of their testimony; and they loved not their lives unto the death."],
['Revelation','Rev','12','012','12',"Therefore rejoice, \emph{ye} heavens, and ye that dwell in them. Woe to the inhabiters of the earth and of the sea! for the devil is come down unto you, having great wrath, because he knoweth that he hath but a short time."],
['Revelation','Rev','12','013','13',"And when the dragon saw that he was cast unto the earth, he persecuted the woman which brought forth the man \emph{child}."],
['Revelation','Rev','12','014','14',"And to the woman were given two wings of a great eagle, that she might fly into the wilderness, into her place, where she is nourished for a time, and times, and half a time, from the face of the serpent."],
['Revelation','Rev','12','015','15',"And the serpent cast out of his mouth water as a flood after the woman, that he might cause her to be carried away of the flood."],
['Revelation','Rev','12','016','16',"And the earth helped the woman, and the earth opened her mouth, and swallowed up the flood which the dragon cast out of his mouth."],
['Revelation','Rev','12','017','17',"And the dragon was wroth with the woman, and went to make war with the remnant of her seed, which keep the commandments of God, and have the testimony of Jesus Christ."]
]


Revelation13 = [
    ['Revelation','Rev','13','001','1',"And I stood upon the sand of the sea, and saw a beast rise up out of the sea, having seven heads and ten horns, and upon his horns ten crowns, and upon his heads the name of blasphemy."],
['Revelation','Rev','13','002','2',"And the beast which I saw was like unto a leopard, and his feet were as \emph{the} \emph{feet} of a bear, and his mouth as the mouth of a lion: and the dragon gave him his power, and his seat, and great authority."],
['Revelation','Rev','13','003','3',"And I saw one of his heads as it were wounded to death; and his deadly wound was healed: and all the world wondered after the beast."],
['Revelation','Rev','13','004','4',"And they worshipped the dragon which gave power unto the beast: and they worshipped the beast, saying, Who \emph{is} like unto the beast? who is able to make war with him?"],
['Revelation','Rev','13','005','5',"And there was given unto him a mouth speaking great things and blasphemies; and power was given unto him to continue forty \emph{and} two months."],
['Revelation','Rev','13','006','6',"And he opened his mouth in blasphemy against God, to blaspheme his name, and his tabernacle, and them that dwell in heaven."],
['Revelation','Rev','13','007','7',"And it was given unto him to make war with the saints, and to overcome them: and power was given him over all kindreds, and tongues, and nations."],
['Revelation','Rev','13','008','8',"And all that dwell upon the earth shall worship him, whose names are not written in the book of life of the Lamb slain from the foundation of the world."],
['Revelation','Rev','13','009','9',"If any man have an ear, let him hear."],
['Revelation','Rev','13','010','10',"He that leadeth into captivity shall go into captivity: he that killeth with the sword must be killed with the sword. Here is the patience and the faith of the saints."],
['Revelation','Rev','13','011','11',"And I beheld another beast coming up out of the earth; and he had two horns like a lamb, and he spake as a dragon."],
['Revelation','Rev','13','012','12',"And he exerciseth all the power of the first beast before him, and causeth the earth and them which dwell therein to worship the first beast, whose deadly wound was healed."],
['Revelation','Rev','13','013','13',"And he doeth great wonders, so that he maketh fire come down from heaven on the earth in the sight of men,"],
['Revelation','Rev','13','014','14',"And deceiveth them that dwell on the earth by \emph{the} \emph{means} \emph{of} those miracles which he had power to do in the sight of the beast; saying to them that dwell on the earth, that they should make an image to the beast, which had the wound by a sword, and did live."],
['Revelation','Rev','13','015','15',"And he had power to give life unto the image of the beast, that the image of the beast should both speak, and cause that as many as would not worship the image of the beast should be killed."],
['Revelation','Rev','13','016','16',"And he causeth all, both small and great, rich and poor, free and bond, to receive a mark in their right hand, or in their foreheads:"],
['Revelation','Rev','13','017','17',"And that no man might buy or sell, save he that had the mark, or the name of the beast, or the number of his name."],
['Revelation','Rev','13','018','18',"Here is wisdom. Let him that hath understanding count the number of the beast: for it is the number of a man; and his number \emph{is} Six hundred threescore \emph{and} six."]
]

Revelation14 = [
    ['Revelation','Rev','14','001','1',"And I looked, and, lo, a Lamb stood on the mount Sion, and with him an hundred forty \emph{and} four thousand, having his Father's name written in their foreheads."],
['Revelation','Rev','14','002','2',"And I heard a voice from heaven, as the voice of many waters, and as the voice of a great thunder: and I heard the voice of harpers harping with their harps:"],
['Revelation','Rev','14','003','3',"And they sung as it were a new song before the throne, and before the four beasts, and the elders: and no man could learn that song but the hundred \emph{and} forty \emph{and} four thousand, which were redeemed from the earth."],
['Revelation','Rev','14','004','4',"These are they which were not defiled with women; for they are virgins. These are they which follow the Lamb whithersoever he goeth. These were redeemed from among men, \emph{being} the firstfruits unto God and to the Lamb."],
['Revelation','Rev','14','005','5',"And in their mouth was found no guile: for they are without fault before the throne of God."],
['Revelation','Rev','14','006','6',"And I saw another angel fly in the midst of heaven, having the everlasting gospel to preach unto them that dwell on the earth, and to every nation, and kindred, and tongue, and people,"],
['Revelation','Rev','14','007','7',"Saying with a loud voice, Fear God, and give glory to him; for the hour of his judgment is come: and worship him that made heaven, and earth, and the sea, and the fountains of waters."],
['Revelation','Rev','14','008','8',"And there followed another angel, saying, Babylon is fallen, is fallen, that great city, because she made all nations drink of the wine of the wrath of her fornication."],
['Revelation','Rev','14','009','9',"And the third angel followed them, saying with a loud voice, If any man worship the beast and his image, and receive \emph{his} mark in his forehead, or in his hand,"],
['Revelation','Rev','14','010','10',"The same shall drink of the wine of the wrath of God, which is poured out without mixture into the cup of his indignation; and he shall be tormented with fire and brimstone in the presence of the holy angels, and in the presence of the Lamb:"],
['Revelation','Rev','14','011','11',"And the smoke of their torment ascendeth up for ever and ever: and they have no rest day nor night, who worship the beast and his image, and whosoever receiveth the mark of his name."],
['Revelation','Rev','14','012','12',"Here is the patience of the saints: here \emph{are} they that keep the commandments of God, and the faith of Jesus."],
['Revelation','Rev','14','013','13',"And I heard a voice from heaven saying unto me, Write, Blessed \emph{are} the dead which die in the Lord from henceforth: Yea, saith the Spirit, that they may rest from their labours; and their works do follow them."],
['Revelation','Rev','14','014','14',"And I looked, and behold a white cloud, and upon the cloud \emph{one} sat like unto the Son of man, having on his head a golden crown, and in his hand a sharp sickle."],
['Revelation','Rev','14','015','15',"And another angel came out of the temple, crying with a loud voice to him that sat on the cloud, Thrust in thy sickle, and reap: for the time is come for thee to reap; for the harvest of the earth is ripe."],
['Revelation','Rev','14','016','16',"And he that sat on the cloud thrust in his sickle on the earth; and the earth was reaped."],
['Revelation','Rev','14','017','17',"And another angel came out of the temple which is in heaven, he also having a sharp sickle."],
['Revelation','Rev','14','018','18',"And another angel came out from the altar, which had power over fire; and cried with a loud cry to him that had the sharp sickle, saying, Thrust in thy sharp sickle, and gather the clusters of the vine of the earth; for her grapes are fully ripe."],
['Revelation','Rev','14','019','19',"And the angel thrust in his sickle into the earth, and gathered the vine of the earth, and cast \emph{it} into the great winepress of the wrath of God."],
['Revelation','Rev','14','020','20',"And the winepress was trodden without the city, and blood came out of the winepress, even unto the horse bridles, by the space of a thousand \emph{and} six hundred furlongs."]
]

Revelation15 = [
    ['Revelation','Rev','15','001','1',"And I saw another sign in heaven, great and marvellous, seven angels having the seven last plagues; for in them is filled up the wrath of God."],
    ['Revelation','Rev','15','002','2',"And I saw as it were a sea of glass mingled with fire: and them that had gotten the victory over the beast, and over his image, and over his mark, \emph{and} over the number of his name, stand on the sea of glass, having the harps of God."],
    ['Revelation','Rev','15','003','3',"And they sing the song of Moses the servant of God, and the song of the Lamb, saying, Great and marvellous \emph{are} thy works, Lord God Almighty; just and true \emph{are} thy ways, thou King of saints."],
    ['Revelation','Rev','15','004','4',"Who shall not fear thee, O Lord, and glorify thy name? for \emph{thou} only \emph{art} holy: for all nations shall come and worship before thee; for thy judgments are made manifest."],     ['Revelation','Rev','15','005','5',"And after that I looked, and, behold, the temple of the tabernacle of the testimony in heaven was opened:"],
    ['Revelation','Rev','15','006','6',"And the seven angels came out of the temple, having the seven plagues, clothed in pure and white linen, and having their breasts girded with golden girdles."],
    ['Revelation','Rev','15','007','7',"And one of the four beasts gave unto the seven angels seven golden vials full of the wrath of God, who liveth for ever and ever."],
    ['Revelation','Rev','15','008','8',"And the temple was filled with smoke from the glory of God, and from his power; and no man was able to enter into the temple, till the seven plagues of the seven angels were fulfilled."]
]

Revelation16 = [
    ['Revelation','Rev','16','001','1',"And I heard a great voice out of the temple saying to the seven angels, Go your ways, and pour out the vials of the wrath of God upon the earth."],
['Revelation','Rev','16','002','2',"And the first went, and poured out his vial upon the earth; and there fell a noisome and grievous sore upon the men which had the mark of the beast, and \emph{upon} them which worshipped his image."],
['Revelation','Rev','16','003','3',"And the second angel poured out his vial upon the sea; and it became as the blood of a dead \emph{man}: and every living soul died in the sea."],
['Revelation','Rev','16','004','4',"And the third angel poured out his vial upon the rivers and fountains of waters; and they became blood."],
['Revelation','Rev','16','005','5',"And I heard the angel of the waters say, Thou art righteous, O Lord, which art, and wast, and shalt be, because thou hast judged thus."],
['Revelation','Rev','16','006','6',"For they have shed the blood of saints and prophets, and thou hast given them blood to drink; for they are worthy."],
['Revelation','Rev','16','007','7',"And I heard another out of the altar say, Even so, Lord God Almighty, true and righteous \emph{are} thy judgments."],
['Revelation','Rev','16','008','8',"And the fourth angel poured out his vial upon the sun; and power was given unto him to scorch men with fire."],
['Revelation','Rev','16','009','9',"And men were scorched with great heat, and blasphemed the name of God, which hath power over these plagues: and they repented not to give him glory."],
['Revelation','Rev','16','010','10',"And the fifth angel poured out his vial upon the seat of the beast; and his kingdom was full of darkness; and they gnawed their tongues for pain,"],
['Revelation','Rev','16','011','11',"And blasphemed the God of heaven because of their pains and their sores, and repented not of their deeds."],
['Revelation','Rev','16','012','12',"And the sixth angel poured out his vial upon the great river Euphrates; and the water thereof was dried up, that the way of the kings of the east might be prepared."],
['Revelation','Rev','16','013','13',"And I saw three unclean spirits like frogs \emph{come} out of the mouth of the dragon, and out of the mouth of the beast, and out of the mouth of the false prophet."],
['Revelation','Rev','16','014','14',"For they are the spirits of devils, working miracles, \emph{which} go forth unto the kings of the earth and of the whole world, to gather them to the battle of that great day of God Almighty."],
['Revelation','Rev','16','015','15',"Behold, I come as a thief. Blessed \emph{is} he that watcheth, and keepeth his garments, lest he walk naked, and they see his shame."],
['Revelation','Rev','16','016','16',"And he gathered them together into a place called in the Hebrew tongue Armageddon."],
['Revelation','Rev','16','017','17',"And the seventh angel poured out his vial into the air; and there came a great voice out of the temple of heaven, from the throne, saying, It is done."],
['Revelation','Rev','16','018','18',"And there were voices, and thunders, and lightnings; and there was a great earthquake, such as was not since men were upon the earth, so mighty an earthquake, \emph{and} so great."],
['Revelation','Rev','16','019','19',"And the great city was divided into three parts, and the cities of the nations fell: and great Babylon came in remembrance before God, to give unto her the cup of the wine of the fierceness of his wrath."],
['Revelation','Rev','16','020','20',"And every island fled away, and the mountains were not found."],
['Revelation','Rev','16','021','21',"And there fell upon men a great hail out of heaven, \emph{every} \emph{stone} about the weight of a talent: and men blasphemed God because of the plague of the hail; for the plague thereof was exceeding great."]
]

Revelation17 = [
    ['Revelation','Rev','17','001','1',"And there came one of the seven angels which had the seven vials, and talked with me, saying unto me, Come hither; I will shew unto thee the judgment of the great whore that sitteth upon many waters:"],
['Revelation','Rev','17','002','2',"With whom the kings of the earth have committed fornication, and the inhabitants of the earth have been made drunk with the wine of her fornication."],
['Revelation','Rev','17','003','3',"So he carried me away in the spirit into the wilderness: and I saw a woman sit upon a scarlet coloured beast, full of names of blasphemy, having seven heads and ten horns."],
['Revelation','Rev','17','004','4',"And the woman was arrayed in purple and scarlet colour, and decked with gold and precious stones and pearls, having a golden cup in her hand full of abominations and filthiness of her fornication:"],
['Revelation','Rev','17','005','5',"And upon her forehead \emph{was} a name written, MYSTERY, BABYLON THE GREAT, THE MOTHER OF HARLOTS AND ABOMINATIONS OF THE EARTH."],
['Revelation','Rev','17','006','6',"And I saw the woman drunken with the blood of the saints, and with the blood of the martyrs of Jesus: and when I saw her, I wondered with great admiration."],
['Revelation','Rev','17','007','7',"And the angel said unto me, Wherefore didst thou marvel? I will tell thee the mystery of the woman, and of the beast that carrieth her, which hath the seven heads and ten horns."],
['Revelation','Rev','17','008','8',"The beast that thou sawest was, and is not; and shall ascend out of the bottomless pit, and go into perdition: and they that dwell on the earth shall wonder, whose names were not written in the book of life from the foundation of the world, when they behold the beast that was, and is not, and yet is."],
['Revelation','Rev','17','009','9',"And here \emph{is} the mind which hath wisdom. The seven heads are seven mountains, on which the woman sitteth."],
['Revelation','Rev','17','010','10',"And there are seven kings: five are fallen, and one is, \emph{and} the other is not yet come; and when he cometh, he must continue a short space."],
['Revelation','Rev','17','011','11',"And the beast that was, and is not, even he is the eighth, and is of the seven, and goeth into perdition."],
['Revelation','Rev','17','012','12',"And the ten horns which thou sawest are ten kings, which have received no kingdom as yet; but receive power as kings one hour with the beast."],
['Revelation','Rev','17','013','13',"These have one mind, and shall give their power and strength unto the beast."],
['Revelation','Rev','17','014','14',"These shall make war with the Lamb, and the Lamb shall overcome them: for he is Lord of lords, and King of kings: and they that are with him \emph{are} called, and chosen, and faithful."],
['Revelation','Rev','17','015','15',"And he saith unto me, The waters which thou sawest, where the whore sitteth, are peoples, and multitudes, and nations, and tongues."],
['Revelation','Rev','17','016','16',"And the ten horns which thou sawest upon the beast, these shall hate the whore, and shall make her desolate and naked, and shall eat her flesh, and burn her with fire."],
['Revelation','Rev','17','017','17',"For God hath put in their hearts to fulfil his will, and to agree, and give their kingdom unto the beast, until the words of God shall be fulfilled."],
['Revelation','Rev','17','018','18',"And the woman which thou sawest is that great city, which reigneth over the kings of the earth."]
]

Revelation18 = [
    ['Revelation','Rev','18','001','1',"And after these things I saw another angel come down from heaven, having great power; and the earth was lightened with his glory."],
['Revelation','Rev','18','002','2',"And he cried mightily with a strong voice, saying, Babylon the great is fallen, is fallen, and is become the habitation of devils, and the hold of every foul spirit, and a cage of every unclean and hateful bird."],
['Revelation','Rev','18','003','3',"For all nations have drunk of the wine of the wrath of her fornication, and the kings of the earth have committed fornication with her, and the merchants of the earth are waxed rich through the abundance of her delicacies."],
['Revelation','Rev','18','004','4',"And I heard another voice from heaven, saying, Come out of her, my people, that ye be not partakers of her sins, and that ye receive not of her plagues."],
['Revelation','Rev','18','005','5',"For her sins have reached unto heaven, and God hath remembered her iniquities."],
['Revelation','Rev','18','006','6',"Reward her even as she rewarded you, and double unto her double according to her works: in the cup which she hath filled fill to her double."],
['Revelation','Rev','18','007','7',"How much she hath glorified herself, and lived deliciously, so much torment and sorrow give her: for she saith in her heart, I sit a queen, and am no widow, and shall see no sorrow."],
['Revelation','Rev','18','008','8',"Therefore shall her plagues come in one day, death, and mourning, and famine; and she shall be utterly burned with fire: for strong \emph{is} the Lord God who judgeth her."],
['Revelation','Rev','18','009','9',"And the kings of the earth, who have committed fornication and lived deliciously with her, shall bewail her, and lament for her, when they shall see the smoke of her burning,"],
['Revelation','Rev','18','010','10',"Standing afar off for the fear of her torment, saying, Alas, alas, that great city Babylon, that mighty city! for in one hour is thy judgment come."],
['Revelation','Rev','18','011','11',"And the merchants of the earth shall weep and mourn over her; for no man buyeth their merchandise any more:"],
['Revelation','Rev','18','012','12',"The merchandise of gold, and silver, and precious stones, and of pearls, and fine linen, and purple, and silk, and scarlet, and all thyine wood, and all manner vessels of ivory, and all manner vessels of most precious wood, and of brass, and iron, and marble,"],
['Revelation','Rev','18','013','13',"And cinnamon, and odours, and ointments, and frankincense, and wine, and oil, and fine flour, and wheat, and beasts, and sheep, and horses, and chariots, and slaves, and souls of men."],
['Revelation','Rev','18','014','14',"And the fruits that thy soul lusted after are departed from thee, and all things which were dainty and goodly are departed from thee, and thou shalt find them no more at all."],
['Revelation','Rev','18','015','15',"The merchants of these things, which were made rich by her, shall stand afar off for the fear of her torment, weeping and wailing,"],
['Revelation','Rev','18','016','16',"And saying, Alas, alas, that great city, that was clothed in fine linen, and purple, and scarlet, and decked with gold, and precious stones, and pearls!"],
['Revelation','Rev','18','017','17',"For in one hour so great riches is come to nought. And every shipmaster, and all the company in ships, and sailors, and as many as trade by sea, stood afar off,"],
['Revelation','Rev','18','018','18',"And cried when they saw the smoke of her burning, saying, What \emph{city} \emph{is} like unto this great city!"],
['Revelation','Rev','18','019','19',"And they cast dust on their heads, and cried, weeping and wailing, saying, Alas, alas, that great city, wherein were made rich all that had ships in the sea by reason of her costliness! for in one hour is she made desolate."],
['Revelation','Rev','18','020','20',"Rejoice over her, \emph{thou} heaven, and \emph{ye} holy apostles and prophets; for God hath avenged you on her."],
['Revelation','Rev','18','021','21',"And a mighty angel took up a stone like a great millstone, and cast \emph{it} into the sea, saying, Thus with violence shall that great city Babylon be thrown down, and shall be found no more at all."],
['Revelation','Rev','18','022','22',"And the voice of harpers, and musicians, and of pipers, and trumpeters, shall be heard no more at all in thee; and no craftsman, of whatsoever craft \emph{he} \emph{be}, shall be found any more in thee; and the sound of a millstone shall be heard no more at all in thee;"],
['Revelation','Rev','18','023','23',"And the light of a candle shall shine no more at all in thee; and the voice of the bridegroom and of the bride shall be heard no more at all in thee: for thy merchants were the great men of the earth; for by thy sorceries were all nations deceived."],
['Revelation','Rev','18','024','24',"And in her was found the blood of prophets, and of saints, and of all that were slain upon the earth."]
]

Revelation19 = [
    ['Revelation','Rev','19','001','1',"And after these things I heard a great voice of much people in heaven, saying, Alleluia; Salvation, and glory, and honour, and power, unto the Lord our God:"],
['Revelation','Rev','19','002','2',"For true and righteous \emph{are} his judgments: for he hath judged the great whore, which did corrupt the earth with her fornication, and hath avenged the blood of his servants at her hand."],
['Revelation','Rev','19','003','3',"And again they said, Alleluia. And her smoke rose up for ever and ever."],
['Revelation','Rev','19','004','4',"And the four and twenty elders and the four beasts fell down and worshipped God that sat on the throne, saying, Amen; Alleluia."],
['Revelation','Rev','19','005','5',"And a voice came out of the throne, saying, Praise our God, all ye his servants, and ye that fear him, both small and great."],
['Revelation','Rev','19','006','6',"And I heard as it were the voice of a great multitude, and as the voice of many waters, and as the voice of mighty thunderings, saying, Alleluia: for the Lord God omnipotent reigneth."],
['Revelation','Rev','19','007','7',"Let us be glad and rejoice, and give honour to him: for the marriage of the Lamb is come, and his wife hath made herself ready."],
['Revelation','Rev','19','008','8',"And to her was granted that she should be arrayed in fine linen, clean and white: for the fine linen is the righteousness of saints."],
['Revelation','Rev','19','009','9',"And he saith unto me, Write, Blessed \emph{are} they which are called unto the marriage supper of the Lamb. And he saith unto me, These are the true sayings of God."],
['Revelation','Rev','19','010','10',"And I fell at his feet to worship him. And he said unto me, See \emph{thou} \emph{do} \emph{it} not: I am thy fellowservant, and of thy brethren that have the testimony of Jesus: worship God: for the testimony of Jesus is the spirit of prophecy."],
['Revelation','Rev','19','011','11',"And I saw heaven opened, and behold a white horse; and he that sat upon him \emph{was} called Faithful and True, and in righteousness he doth judge and make war."],
['Revelation','Rev','19','012','12',"His eyes \emph{were} as a flame of fire, and on his head \emph{were} many crowns; and he had a name written, that no man knew, but he himself."],
['Revelation','Rev','19','013','13',"And he \emph{was} clothed with a vesture dipped in blood: and his name is called The Word of God."],
['Revelation','Rev','19','014','14',"And the armies \emph{which} \emph{were} in heaven followed him upon white horses, clothed in fine linen, white and clean."],
['Revelation','Rev','19','015','15',"And out of his mouth goeth a sharp sword, that with it he should smite the nations: and he shall rule them with a rod of iron: and he treadeth the winepress of the fierceness and wrath of Almighty God."],
['Revelation','Rev','19','016','16',"And he hath on \emph{his} vesture and on his thigh a name written, KING OF KINGS, AND LORD OF LORDS."],
['Revelation','Rev','19','017','17',"And I saw an angel standing in the sun; and he cried with a loud voice, saying to all the fowls that fly in the midst of heaven, Come and gather yourselves together unto the supper of the great God;"],
['Revelation','Rev','19','018','18',"That ye may eat the flesh of kings, and the flesh of captains, and the flesh of mighty men, and the flesh of horses, and of them that sit on them, and the flesh of all \emph{men,} \emph{both} free and bond, both small and great."],
['Revelation','Rev','19','019','19',"And I saw the beast, and the kings of the earth, and their armies, gathered together to make war against him that sat on the horse, and against his army."],
['Revelation','Rev','19','020','20',"And the beast was taken, and with him the false prophet that wrought miracles before him, with which he deceived them that had received the mark of the beast, and them that worshipped his image. These both were cast alive into a lake of fire burning with brimstone."],
['Revelation','Rev','19','021','21',"And the remnant were slain with the sword of him that sat upon the horse, which \emph{sword} proceeded out of his mouth: and all the fowls were filled with their flesh."]
]

Revelation20 = [
    ['Revelation','Rev','20','001','1',"And I saw an angel come down from heaven, having the key of the bottomless pit and a great chain in his hand."],
    ['Revelation','Rev','20','002','2',"And he laid hold on the dragon, that old serpent, which is the Devil, and Satan, and bound him a thousand years,"],
    ['Revelation','Rev','20','003','3',"And cast him into the bottomless pit, and shut him up, and set a seal upon him, that he should deceive the nations no more, till the thousand years should be fulfilled: and after that he must be loosed a little season."],
    ['Revelation','Rev','20','004','4',"And I saw thrones, and they sat upon them, and judgment was given unto them: and \emph{I} \emph{saw} the souls of them that were beheaded for the witness of Jesus, and for the word of God, and which had not worshipped the beast, neither his image, neither had received \emph{his} mark upon their foreheads, or in their hands; and they lived and reigned with Christ a thousand years."],
    ['Revelation','Rev','20','005','5',"But the rest of the dead lived not again until the thousand years were finished. This \emph{is} the first resurrection."],
    ['Revelation','Rev','20','006','6',"Blessed and holy \emph{is} he that hath part in the first resurrection: on such the second death hath no power, but they shall be priests of God and of Christ, and shall reign with him a thousand years."],
    ['Revelation','Rev','20','007','7',"And when the thousand years are expired, Satan shall be loosed out of his prison,"],
    ['Revelation','Rev','20','008','8',"And shall go out to deceive the nations which are in the four quarters of the earth, Gog and Magog, to gather them together to battle: the number of whom \emph{is} as the sand of the sea."],
    ['Revelation','Rev','20','009','9',"And they went up on the breadth of the earth, and compassed the camp of the saints about, and the beloved city: and fire came down from God out of heaven, and devoured them."],
    ['Revelation','Rev','20','010','10',"And the devil that deceived them was cast into the lake of fire and brimstone, where the beast and the false prophet \emph{are}, and shall be tormented day and night for ever and ever."],
    ['Revelation','Rev','20','011','11',"And I saw a great white throne, and him that sat on it, from whose face the earth and the heaven fled away; and there was found no place for them."],
    ['Revelation','Rev','20','012','12',"And I saw the dead, small and great, stand before God; and the books were opened: and another book was opened, which is \emph{the} \emph{book} of life: and the dead were judged out of those things which were written in the books, according to their works."],
    ['Revelation','Rev','20','013','13',"And the sea gave up the dead which were in it; and death and hell delivered up the dead which were in them: and they were judged every man according to their works."],
    ['Revelation','Rev','20','014','14',"And death and hell were cast into the lake of fire. This is the second death."],
    ['Revelation','Rev','20','015','15',"And whosoever was not found written in the book of life was cast into the lake of fire."]
]
    
Revelation21 = [
    ['Revelation','Rev','21','001','1',"And I saw a new heaven and a new earth: for the first heaven and the first earth were passed away; and there was no more sea."],
['Revelation','Rev','21','002','2',"And I John saw the holy city, new Jerusalem, coming down from God out of heaven, prepared as a bride adorned for her husband."],
['Revelation','Rev','21','003','3',"And I heard a great voice out of heaven saying, Behold, the tabernacle of God \emph{is} with men, and he will dwell with them, and they shall be his people, and God himself shall be with them, \emph{and} \emph{be} their God."],
['Revelation','Rev','21','004','4',"And God shall wipe away all tears from their eyes; and there shall be no more death, neither sorrow, nor crying, neither shall there be any more pain: for the former things are passed away."],
['Revelation','Rev','21','005','5',"And he that sat upon the throne said, Behold, I make all things new. And he said unto me, Write: for these words are true and faithful."],
['Revelation','Rev','21','006','6',"And he said unto me, It is done. I am Alpha and Omega, the beginning and the end. I will give unto him that is athirst of the fountain of the water of life freely."],
['Revelation','Rev','21','007','7',"He that overcometh shall inherit all things; and I will be his God, and he shall be my son."],
['Revelation','Rev','21','008','8',"But the fearful, and unbelieving, and the abominable, and murderers, and whoremongers, and sorcerers, and idolaters, and all liars, shall have their part in the lake which burneth with fire and brimstone: which is the second death."],
['Revelation','Rev','21','009','9',"And there came unto me one of the seven angels which had the seven vials full of the seven last plagues, and talked with me, saying, Come hither, I will shew thee the bride, the Lamb's wife."],
['Revelation','Rev','21','010','10',"And he carried me away in the spirit to a great and high mountain, and shewed me that great city, the holy Jerusalem, descending out of heaven from God,"],
['Revelation','Rev','21','011','11',"Having the glory of God: and her light \emph{was} like unto a stone most precious, even like a jasper stone, clear as crystal;"],
['Revelation','Rev','21','012','12',"And had a wall great and high, \emph{and} had twelve gates, and at the gates twelve angels, and names written thereon, which are \emph{the} \emph{names} of the twelve tribes of the children of Israel:"],
['Revelation','Rev','21','013','13',"On the east three gates; on the north three gates; on the south three gates; and on the west three gates."],
['Revelation','Rev','21','014','14',"And the wall of the city had twelve foundations, and in them the names of the twelve apostles of the Lamb."],
['Revelation','Rev','21','015','15',"And he that talked with me had a golden reed to measure the city, and the gates thereof, and the wall thereof."],
['Revelation','Rev','21','016','16',"And the city lieth foursquare, and the length is as large as the breadth: and he measured the city with the reed, twelve thousand furlongs. The length and the breadth and the height of it are equal."],
['Revelation','Rev','21','017','17',"And he measured the wall thereof, an hundred \emph{and} forty \emph{and} four cubits, \emph{according} \emph{to} the measure of a man, that is, of the angel."],
['Revelation','Rev','21','018','18',"And the building of the wall of it was \emph{of} jasper: and the city \emph{was} pure gold, like unto clear glass."],
['Revelation','Rev','21','019','19',"And the foundations of the wall of the city \emph{were} garnished with all manner of precious stones. The first foundation \emph{was} jasper; the second, sapphire; the third, a chalcedony; the fourth, an emerald;"],
['Revelation','Rev','21','020','20',"The fifth, sardonyx; the sixth, sardius; the seventh, chrysolite; the eighth, beryl; the ninth, a topaz; the tenth, a chrysoprasus; the eleventh, a jacinth; the twelfth, an amethyst."],
['Revelation','Rev','21','021','21',"And the twelve gates \emph{were} twelve pearls; every several gate was of one pearl: and the street of the city \emph{was} pure gold, as it were transparent glass."],
['Revelation','Rev','21','022','22',"And I saw no temple therein: for the Lord God Almighty and the Lamb are the temple of it."],
['Revelation','Rev','21','023','23',"And the city had no need of the sun, neither of the moon, to shine in it: for the glory of God did lighten it, and the Lamb \emph{is} the light thereof."],
['Revelation','Rev','21','024','24',"And the nations of them which are saved shall walk in the light of it: and the kings of the earth do bring their glory and honour into it."],
['Revelation','Rev','21','025','25',"And the gates of it shall not be shut at all by day: for there shall be no night there."],
['Revelation','Rev','21','026','26',"And they shall bring the glory and honour of the nations into it."],
['Revelation','Rev','21','027','27',"And there shall in no wise enter into it any thing that defileth, neither \emph{whatsoever} worketh abomination, or \emph{maketh} a lie: but they which are written in the Lamb's book of life."]
]
    
Revelation22 = [
    ['Revelation','Rev','22','001','1',"And he shewed me a pure river of water of life, clear as crystal, proceeding out of the throne of God and of the Lamb."],
    ['Revelation','Rev','22','002','2',"In the midst of the street of it, and on either side of the river, \emph{was} \emph{there} the tree of life, which bare twelve \emph{manner} \emph{of} fruits, \emph{and} yielded her fruit every month: and the leaves of the tree \emph{were} for the healing of the nations."],
    ['Revelation','Rev','22','003','3',"And there shall be no more curse: but the throne of God and of the Lamb shall be in it; and his servants shall serve him:"],
    ['Revelation','Rev','22','004','4',"And they shall see his face; and his name \emph{shall} \emph{be} in their foreheads."],
    ['Revelation','Rev','22','005','5',"And there shall be no night there; and they need no candle, neither light of the sun; for the Lord God giveth them light: and they shall reign for ever and ever."],
    ['Revelation','Rev','22','006','6',"And he said unto me, These sayings \emph{are} faithful and true: and the Lord God of the holy prophets sent his angel to shew unto his servants the things which must shortly be done."],
    ['Revelation','Rev','22','007','7',"Behold, I come quickly: blessed \emph{is} he that keepeth the sayings of the prophecy of this book."],
    ['Revelation','Rev','22','008','8',"And I John saw these things, and heard \emph{them}. And when I had heard and seen, I fell down to worship before the feet of the angel which shewed me these things."],
    ['Revelation','Rev','22','009','9',"Then saith he unto me, See \emph{thou} \emph{do} \emph{it} not: for I am thy fellowservant, and of thy brethren the prophets, and of them which keep the sayings of this book: worship God."],
    ['Revelation','Rev','22','010','10',"And he saith unto me, Seal not the sayings of the prophecy of this book: for the time is at hand."],
    ['Revelation','Rev','22','011','11',"He that is unjust, let him be unjust still: and he which is filthy, let him be filthy still: and he that is righteous, let him be righteous still: and he that is holy, let him be holy still."],
    ['Revelation','Rev','22','012','12',"And, behold, I come quickly; and my reward \emph{is} with me, to give every man according as his work shall be."],
    ['Revelation','Rev','22','013','13',"I am Alpha and Omega, the beginning and the end, the first and the last."],
    ['Revelation','Rev','22','014','14',"Blessed \emph{are} they that do his commandments, that they may have right to the tree of life, and may enter in through the gates into the city."],
    ['Revelation','Rev','22','015','15',"For without \emph{are} dogs, and sorcerers, and whoremongers, and murderers, and idolaters, and whosoever loveth and maketh a lie."],
    ['Revelation','Rev','22','016','16',"I Jesus have sent mine angel to testify unto you these things in the churches. I am the root and the offspring of David, \emph{and} the bright and morning star."],
    ['Revelation','Rev','22','017','17',"And the Spirit and the bride say, Come. And let him that heareth say, Come. And let him that is athirst come. And whosoever will, let him take the water of life freely."],
    ['Revelation','Rev','22','018','18',"For I testify unto every man that heareth the words of the prophecy of this book, If any man shall add unto these things, God shall add unto him the plagues that are written in this book:"],
    ['Revelation','Rev','22','019','19',"And if any man shall take away from the words of the book of this prophecy, God shall take away his part out of the book of life, and out of the holy city, and \emph{from} the things which are written in this book."],
    ['Revelation','Rev','22','020','20',"He which testifieth these things saith, Surely I come quickly. Amen. Even so, come, Lord Jesus."],
    ['Revelation','Rev','22','021','21',"The grace of our Lord Jesus Christ \emph{be} with you all. Amen."]
]
    

def main():
    ProcessVerses('Revelation10',Revelation10)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And I saw another mighty angel come down from heaven, clothed with a cloud: and a rainbow \emph{was} upon his head, and his face \emph{was} as it were the sun, and his feet as pillars of fire:}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And he had in his hand a little book open: and he set his right foot upon the sea, and \emph{his} left \emph{foot} on the earth,}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And cried with a loud voice, as \emph{when} a lion roareth: and when he had cried, seven thunders uttered their voices.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And when the seven thunders had uttered their voices, I was about to write: and I heard a voice from heaven saying unto me, Seal up those things which the seven thunders uttered, and write them not.}
[5] \textcolor[cmyk]{0.99998,1,0,